In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
!pip install datasets
!pip install peft
!pip install -U bitsandbytes


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [ ]:
from datasets import load_dataset



In [ ]:
import torch
import io
import pickle
import pandas as pd
from transformers import BlipProcessor, BlipForQuestionAnswering
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


## Load Model HuggingFaceM4/Idefics3-8B-Llama3

In [ ]:
import torch
from peft import LoraConfig
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics2ForConditionalGeneration

DEVICE = "cuda:0"
USE_LORA = False
USE_QLORA = True

processor = AutoProcessor.from_pretrained(
    "HuggingFaceM4/Idefics3-8B-Llama3",
    do_image_splitting=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, BitsAndBytesConfig, AutoModelForImageTextToText,AutoModelForVision2Seq
from peft import LoraConfig

if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=4,
        lora_alpha=4,
        lora_dropout=0.1,
        target_modules='.*(text_model|modality_projection|perceiver_resampler).*(down_proj|gate_proj|up_proj|k_proj|q_proj|v_proj|o_proj).*$',
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )

    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
    else:
        bnb_config = None

    model = AutoModelForVision2Seq.from_pretrained(
        "HuggingFaceM4/Idefics3-8B-Llama3",
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
    )

    # Add LoRA configuration to the model
    model.add_adapter(lora_config)
    model.enable_adapters()
else:
    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/Idefics3-8B-Llama3",
        torch_dtype=torch.float16,
        _attn_implementation="flash_attention_2",
    ).to(DEVICE)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
!pip install -U datasets
!pip install fsspec==2023.9.2

In [ ]:
from datasets import load_dataset
dataset = load_dataset("flaviagiammarino/vqa-rad")



In [ ]:
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

# Rename the splits
train_dataset = train_test_split['train']
valid_dataset = train_test_split['test']
datasets = {
    'train': train_dataset,
    'valid': valid_dataset
}

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(valid_dataset)}")

Train size: 1434
Validation size: 359


In [ ]:
from peft import LoraConfig
from datasets import load_dataset
from torch.utils.data import Dataset
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score, recall_score
from torch.utils.data import Dataset
import random

class VQADataset(Dataset):
    def __init__(self, hf_dataset, processor):
        self.dataset = hf_dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        image = self.dataset[idx]['image']  # This is a PIL image

        return {
            "image": image,  # Directly use the PIL image
            "query": {"en": question},
            "answers": [answer]
        }

# Define the MyDataCollator class
class MyDataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, examples):
        texts = []
        images = []
        for example in examples:
            image = example["image"]  # Directly use the PIL image
            question = example["query"]['en']
            answer = random.choice(example["answers"])

            # Create messages as before
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Answer briefly."},
                        {"type": "image"},
                        {"type": "text", "text": question}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": answer}
                    ]
                }
            ]
            text = self.processor.apply_chat_template(messages, add_generation_prompt=False)
            texts.append(text.strip())
            images.append(image)

        # Process texts and images using the processor
        batch = self.processor(text=texts, images=images, return_tensors="pt", padding=True)

        # Create labels for training
        labels = batch["input_ids"].clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = -100  # Ignore padding in loss calculation
        batch["labels"] = labels

        return batch



dataset1 = load_dataset("flaviagiammarino/vqa-rad")

train_dataset = VQADataset(hf_dataset=datasets['train'], processor=processor)
test_dataset = VQADataset(hf_dataset=dataset1['test'], processor=processor)
valid_dataset = VQADataset(hf_dataset=datasets['valid'], processor=processor)


# Data collator for batching
data_collator = MyDataCollator(processor=processor)


In [ ]:
from transformers import TrainingArguments, Trainer

from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="IDEFICS_llama1",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=100,  # Increased evaluation frequency
    save_steps=100,
    max_steps=300,
    logging_steps=5,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["labels"],
    report_to="none",
    optim="paged_adamw_8bit",
    weight_decay=0.05,
    # Add early stopping callback
    # load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)





In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,

)

trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,0.105900,0.116991


Step,Training Loss,Validation Loss
100,0.105900,0.116991
200,0.071200,0.112436
300,0.064400,0.110584


TrainOutput(global_step=300, training_loss=0.270340869029363, metrics={'train_runtime': 2934.5679, 'train_samples_per_second': 0.818, 'train_steps_per_second': 0.102, 'total_flos': 2.2600737168309696e+16, 'train_loss': 0.270340869029363, 'epoch': 1.6694560669456067})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,LlavaProcessor

# Define save path
save_directory = "/content/drive/MyDrive/VQAMed/VQA/models/R4Idefics3-8B-LlamafirMed300"


from transformers import AutoConfig

config = AutoConfig.from_pretrained("HuggingFaceM4/Idefics3-8B-Llama3")
model.config.save_pretrained(save_directory)
model.save_pretrained(save_directory)
processor = LlavaProcessor.from_pretrained("HuggingFaceM4/Idefics3-8B-Llama3")
processor.save_pretrained(save_directory)

print(f"✅ Model saved at: {save_directory}")

print(f"✅ Model saved at: {save_directory}")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Model saved at: /content/drive/MyDrive/VQAMed/VQA/models/R4Idefics3-8B-LlamafirMed
✅ Model saved at: /content/drive/MyDrive/VQAMed/VQA/models/R4Idefics3-8B-LlamafirMed


In [ ]:
from accelerate.utils import release_memory
# ...

release_memory(model)

[None]

In [ ]:
results = trainer.evaluate()
results

{'eval_loss': 0.11058422923088074,
 'eval_runtime': 200.4896,
 'eval_samples_per_second': 1.791,
 'eval_steps_per_second': 1.791,
 'epoch': 1.6694560669456067}

In [ ]:
test_results = trainer.evaluate(test_dataset)

print(test_results)

{'eval_loss': 0.11989174038171768, 'eval_runtime': 251.9129, 'eval_samples_per_second': 1.79, 'eval_steps_per_second': 1.79, 'epoch': 1.6694560669456067}


## Push model to hugging face

In [ ]:
from huggingface_hub import HfApi

save_directory = "/content/drive/MyDrive/VQAMed/VQA/models/R4Idefics3-8B-LlamafirMed300"
repo_name = "R4llaMed3009Jj"
user_name = "Dtarget"

api = HfApi()

api.create_repo(repo_id=f"{user_name}/{repo_name}", private=True, repo_type="model")

api.upload_folder(
    folder_path=save_directory,
    repo_id=f"{user_name}/{repo_name}",
    path_in_repo=""
)

print(f"✅ Model uploaded to Hugging Face privately: https://huggingface.co/{user_name}/{repo_name}")


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

✅ Model uploaded to Hugging Face privately: https://huggingface.co/Dtarget/R4llaMed9Jj


In [ ]:
from accelerate.utils import release_memory
# ...

release_memory(model)

[None]